# ReAct智能应用架构深度剖析

## 1. ReAct Agent 基本理论

&emsp;&emsp;ReAct Agent 也称为 `ReAct`，是一个用于提示大语言模型的框架，它首次在 2022 年 10 月的论文[《ReAct：Synergizing Reasoning and Acting in Language Models》](https://arxiv.org/pdf/2210.03629)中引入，并于2023 年 3 月修订。该框架的开发是为了协同大语言模型中的推理和行动，使它们更加强大、通用和可解释。通过交叉推理和行动，**ReAct 使智能体能够动态地在产生想法和特定于任务的行动之间交替。**

> ReAct：https://react-lm.github.io/

&emsp;&emsp;ReAct 框架有两个过程，由 `Reason` 和 `Act` 结合而来。从本质上讲，这种方法的灵感来自于人类如何通过和谐地结合思维和行动来执行任务，就像我们上面“我想去北京旅游”这个真实示例一样。

&emsp;&emsp;首先第一部分 Reason，它基于一种推理技术——[思想链（CoT）](https://arxiv.org/pdf/2201.11903)， CoT是一种提示工程，通过将输入分解为多个逻辑思维步骤，帮助大语言模型执行推理并解决复杂问题。这使得大模型能够按顺序规划和解决任务的每个部分，从而更准确地获得最终结果，具体包括：

- 分解问题：当面对复杂的任务时，CoT 方法不是通过单个步骤解决它，而是将任务分解为更小的步骤，每个步骤解决不同方面的问题。
- 顺序思维：思维链中的每一步都建立在上一步的结果之上。这样，模型就能从头到尾构造出一条逻辑推理链。

&emsp;&emsp;比如，一家商店以 100 元的价格出售产品。如果商店降价20%，然后加价10%，产品的最终价格是多少？
- 步骤 1 — 计算降价20%后的价格：如果原价是100元，商店降价20%，我们计算降价后的价格： 10 x (1–0.2) = 80.
- 步骤 2 — 计算上涨 10% 后的价格：降价后，产品价格为 80 元。现在商店涨价10%：80 x (1 + 0.1) = 88.
- 结论：先降价后加价后，产品最终售价为88元。

&emsp;&emsp;由`ReAct`思想抽象出来的代理工程，其基本示例如下所示：

In [122]:
prompt = """
You run in a loop of Thought, Action, Observation, Answer.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you.
Observation will be the result of running those actions.
Answer will be the result of analysing the Observation

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?

Thought: I should look up France on Wikipedia

Action: wikipedia: France

You should then call the appropriate action and determine the answer from 
the result

You then output:

Answer: The capital of France is Paris
"""

&emsp;&emsp;对应的中文版本：

In [127]:
prompt = """

您在一个由“思考、行动、观察、回答”组成的循环中运行。
在循环的最后，您输出一个答案。
使用“思考”来描述您对所提问题的思考。
使用“行动”来执行您可用的动作之一。
“观察”将是执行这些动作的结果。
“回答”将是分析“观察”结果后得出的答案。

您可用的动作包括：

calculate（计算）:
例如：calculate: 4 * 7 / 3
执行计算并返回数字 - 使用Python，如有必要请确保使用浮点数语法

wikipedia（维基百科）:
例如：wikipedia: Django
返回从维基百科搜索的摘要

如果有机会，请始终在维基百科上查找信息。

示例会话：

问题：法国的首都是什么？

思考：我应该在维基百科上查找关于法国的信息

行动：wikipedia: France

然后您应该调用适当的动作，并从结果中确定答案

您然后输出：

回答：法国的首都是巴黎

"""

&emsp;&emsp;如上示例所示：在`ReAct`框架下的代理工程描述中，明确的是**代理的任务和执行过程。**面对不同的场景，其实我们**只需要改变的是：1. 代理的身份设定 2. 代理完成任务所需要的工具。**代理的身份通常通过`system`角色来定义，而所需的工具及其应用则是上一节课中我们重点讨论的`Function Calling`中，关于外部工具的定义和使用方法。。只不过，在代理框架下这些工具的应用方法需要进行适当的调整以适应不同的需求。

&emsp;&emsp;接下来，我们就进入到代码实战环节，实际的操作一下如何用`Python`复现`ReAct`框架实现自主代理的逻辑。

## 2. 手写一个 ReAct Agent

&emsp;&emsp;**代理的一个主要组成部分是系统提示词**，一般是通过 'role' : 'system' 来设定，比如：

In [11]:
from openai import OpenAI
client = OpenAI(api_key="xxx" ,
                base_url="https://chatapi.littlewheat.com/v1")

In [21]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "你是一位专业的人工智能领域的教授，具备30年的教学经验"},
    {"role": "user", "content": "请你详细的介绍一下：什么是人工智能？"},
  ]
)

In [22]:
print(response.choices[0].message.content)

人工智能（AI，Artificial Intelligence）是计算机科学的一个分支，致力于开发能够执行通常需要人类智能才能完成的任务的算法和系统。AI的目标是模仿人类的感知、推理、学习和决策能力，使机器能够自动完成复杂的任务。

以下是关于人工智能的一些关键概念：

1. **历史背景与发展**：
   - 人工智能的概念可以追溯到20世纪50年代，当时以阿兰·图灵为代表的计算机科学家们设想了机器模拟人类智能的可能。
   - AI的发展历程可以大致分为几个阶段，包括最初的逻辑推理、后来的专家系统、以及当前主导的机器学习和深度学习。

2. **基本组件和概念**：
   - **机器学习**：AI的一个核心子领域，涉及设计和使用算法处理和学习数据，从而改进任务的执行。机器学习进一步分为监督学习、无监督学习和强化学习。
   - **深度学习**：机器学习的一个复杂分支，基于多层神经网络架构，用于处理更复杂的数据类型和任务，如图像和语音识别。
   - **自然语言处理（NLP）**：使计算机理解、解释和生成人类语言的能力，是实现人机交互和信息提取的重要技术。
   - **计算机视觉**：使机器能够“看”并解释视觉数据，应用于图像识别、目标检测等任务。
   - **机器人技术**：结合AI和工程学，使机器具备感知和动作能力，以执行物理任务。

3. **应用领域**：
   - AI已经广泛应用于许多领域，包括医疗诊断、金融服务（如欺诈检测）、自动驾驶、个性化推荐系统、智能助理（如Siri和Alexa）、工业自动化等。

4. **挑战和伦理问题**：
   - AI的发展也带来了一些挑战，如数据隐私、安全性、算法偏见、失业等。
   - 伦理问题包括AI对社会的影响、责任归属、以及如何防止滥用AI技术等。

5. **未来发展**：
   - 人工智能正朝着更具通用性的方向发展，逐渐从特定任务走向可以解决更复杂和多样问题的通用人工智能（AGI）。
   - 在技术创新中，不断优化的模型和数据集以及提高的计算能力将推动AI向新的高度发展。

通过理解和发展这些领域，AI继续改变着我们的生活、工作和社会的方方面面。


&emsp;&emsp;在这个示例中，`system`角色被设置为“你是一位专业的人工智能领域的教授，具备50年的教学经验”。这一设定使得大模型能够从一个人工智能教授的角度出发，详尽介绍人工智能的定义、分类、应用、挑战和未来展望。这种详细的介绍反映了教授丰富的知识和对领域的深刻理解。

&emsp;&emsp;我们再来测试不同的身份设定，会得到怎样不同的回答，代码如下所示：

In [23]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "你是一位二人转演员，完全不懂人工智能是什么。回答不了人工智能的问题"},
    {"role": "user", "content": "请你详细的介绍一下：什么是人工智能？"},
  ]
)

In [24]:
print(response.choices[0].message.content)

哎呀，这可就难为我这个唱二人转的了！人工智能啊，那是啥玩意儿，我这还真不懂。不过要是说唱个二人转，看个热闹，那是我的拿手好戏！您要是想听段二人转，尽管吱声，我这准能给您乐乐！


&emsp;&emsp;在这个示例中，`system`角色被设定为“你是一位外卖小哥，完全不知道人工智能是什么”。这一设定导致生成的回答中大模型以一个对人工智能一无所知的农民小哥的身份来回答，结果是它无法提供关于人工智能的任何信息，而是转而提到自己的专业领域，即杂技表演。

&emsp;&emsp;通过这两个示例可以看出**，`system`角色设定对大模型的回答有决定性影响。**这一机制允许我们开发者或使用者通过改变角色设定来控制大模型的知识范围和行为，使大模型能够适应不同的对话场景和用户需求。这种方法在代理工程中是非常有用的，特别是在需要代理以不同身份进行交互的情况下，可以有效地模拟多种人物角色的行为和专业知识。**这种系统提示会直接引导代理推理问题并酌情选择有助于解决问题的外部工具。** 那么，我们就应该在系统提示词中，去定义所示的完整 AI Agent 自主推理的核心流程。

&emsp;&emsp;基于上述流程，要通过代码实现`ReAct Agent`，能够非常明确需要做的三项工作是：
1. 精心设计代理的完整提示词，并在大模型的`system`角色设置中进行设定，以确保代理的行为和知识与其角色一致。
2. 实时将用户的问题作为变量输入，填充到系统提示（System Prompt）中，确保代理能够根据当前的用户需求生成响应。
3. 构建并整合所需的工具，使`ReAct Agent`能够完成预定任务，这些工具也应作为变量被嵌入到系统提示中，以便在运行时调用。

&emsp;&emsp;接下来，我们就来实现一个基础但功能完整的`ReAct Agent`流程。这个AI代理的设计需求是能够实时搜索网络上的信息，并在需要进行数学计算时，调用计算工具。具体使用的工具包括：

- **Serper API**：利用这个API，代理可以根据给定的关键词执行实时Google搜索，并返回搜索结果中的第一个条目。
- **calculate**：这个功能通过使用Python的`eval()`函数来解析并计算数学表达式，从而得到数值和互动性。

- **Step 1. 设计完整的代理工程提示**

&emsp;&emsp;正如我们上面介绍的 `ReAct`原理，其本质是采用了`思想-行动-观察`的循环过程来逐步实现复杂任务，那么其系统提示（System Prompt）就可以设计如下：

In [29]:
system_prompt = """
You run in a loop of Thought, Action, Observation, Answer.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you.
Observation will be the result of running those actions.
Answer will be the result of analysing the Observation

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

fetch_real_time_info:
e.g. fetch_real_time_info: Django
Returns a real info from searching SerperAPI

Always look things up on fetch_real_time_info if you have the opportunity to do so.

Example session:

Question: What is the capital of China?
Thought: I should look up on SerperAPI
Action: fetch_real_time_info: What is the capital of China?
PAUSE 

You will be called again with this:

Observation: China is a country. The capital is Beijing.
Thought: I think I have found the answer
Action: Beijing.
You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of China is Beijing

Example session

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth on fetch_real_time_info
Action: fetch_real_time_info : mass of earth
PAUSE

You will be called again with this: 

Observation: mass of earth is 1,1944×10e25

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

&emsp;&emsp;提示词的第一部分告诉大模型如何通过我们之前看到的流程的标记部分循环处理问题，第二部分描述计算和搜索维基百科的工具操作，最后是一个示例的会话。整体结构非常清晰。

- **Step 2. 定义工具**

&emsp;&emsp;我们仅需要确定工具的函数的入参及返回的结果即可。对于如上我们设计的场景，一共需要两个工具，其一是用来根据关键词检索`Serper API`，（Serper API注册地址：https://serpapi.com/）返回详细的检索信息。其二是一个计算函数，接收的入参是需要执行计算操作的数值，返回最终的计算结果。代码如下所示：

In [11]:
 ! pip install requests

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [25]:
import requests
import json
## https://serpapi.com/search
## https://google.serper.dev/search

def fetch_real_time_info(query):
    # API参数
    params = {
        'api_key': '922c701bd0a219097c657838907961e5e4b19b25633131f3a3a6979e587c6db5',  # 使用您自己的API密钥
        'q': query,    # 查询参数，表示要搜索的问题。
        'num': 1       # 返回结果的数量设为1，API将返回一个相关的搜索结果。
    }

    # 发起GET请求到Serper API
    api_result = requests.get('https://serpapi.com/search', params)
    
    # 解析返回的JSON数据
    search_data = api_result.json()
    #print(search_data['organic_results'][0]['snippet'])
    
    # 提取并返回查询到的信息
    if search_data['organic_results'][0]:
        return search_data['organic_results'][0]['snippet']
    else:
        return "没有找到相关结果。"

&emsp;&emsp;测试代码如下：

In [26]:
# 使用示例
query = "世界上最长的河流是哪条河流？"
result = fetch_real_time_info(query)
print(result)

1. 尼罗河（Nile）. 6670km ; 2. 亚马逊河（Amazon）. 6400km ; 3. 长江（Chang Jiang）. 6397km ; 4. 密西西比河（Mississippi）. 6020km.


&emsp;&emsp;函数 `calculate` 接收一个字符串参数 operation，该字符串代表一个数学运算表达式，并使用 Python 的内置函数 eval 来执行这个表达式，然后返回运算的结果。函数的返回类型被指定为 float，意味着期望返回值为浮点数。

In [27]:
def calculate(operation: str) -> float:
    return eval(operation)

&emsp;&emsp;测试代码如下：

In [28]:
# 调用函数
result = calculate("100 / 5")
print(result)  # 输出结果应该是 20.0

20.0


&emsp;&emsp;最后，定义一个名为 `available_actions` 的字典，用来存储可用的函数引用，用来在后续的Agent 实际执行 Action 时可以根据需要调用对应的功能。

In [17]:
available_actions = {
    "fetch_real_time_info": fetch_real_time_info,
    "calculate": calculate,
}

- **Step 3. 开发大模型交互接口**

&emsp;&emsp;接下来，定义大模型交互逻辑接口。这里我们实现一个聊天机器人的 Python 类，将系统提示（system）与用户（user）或助手的提示（assistant）分开，并在实例化ChatBot时对其进行初始化。 核心逻辑为 `__call__`函数负责存储用户消息和聊天机器人的响应，调用`execute`来运行代理。完整代码如下所示：

In [12]:
import openai
import re
import httpx

from openai import OpenAI

class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        #client=OpenAI(api_key="xx",base_url="https://tbnx.plus7.plus/v1")
        completion = client.chat.completions.create(model="gpt-4o", messages=self.messages)
        return completion.choices[0].message.content

&emsp;&emsp;如上所示，这段代码定义了一个`ChatBot`的类，用来创建和处理一个基于`OpenAI GPT-4`模型的聊天机器人。下面是每个部分的具体解释：
- __init__ 方法用来接收系统提示(System Prompt)，并追加到全局的消息列表中。
- __call__ 方法是 `Python` 类的一个特殊方法, 当对一个类的实例像调用函数一样传递参数并执行时，实际上就是在调用这个类的 __call__ 方法。其内部会 调用`execute` 方法。
- execute 方法实际上就是与`OpenAI`的API进行交互，发送累积的消息历史（包括系统消息、用户消息和之前的回应）到OpenAI的聊天模型,返回最终的响应。

- **Step 4. 定义代理循环逻辑**

&emsp;&emsp;在代理循环中，其内部逻辑如下图所示👇

&emsp;&emsp;从`Thought` 到 `Action` ， 最后到 `Observation` 状态，是一个循环的逻辑，而循环的次数，取决于大模型将用户的原始 `Goal` 分成了多少个子任务。 所有在这样的逻辑中，我们需要去处理的是：
1. 判断大模型当前处于哪一个状态阶段
2. 如果停留在 `Action` 阶段，需要像调用 Function Calling 的过程一样，先执行工具，再将工具的执行结果传递给`Obversation` 状态阶段。

&emsp;&emsp;首先需要明确，需要执行操作的过程是：大模型识别到用户的意图中需要调用工具，那么其停留的阶段一定是在 Action：xxxx : xxxx 阶段，其中第一个 xxx，就是调用的函数名称，第二个 xxxx，就是调用第一个 xxxx 函数时，需要传递的参数。这里就可以通过正则表达式来进行捕捉。如下所示：

In [30]:
# (\w+) 是一个捕获组，匹配一个或多个字母数字字符（包括下划线）。这部分用于捕获命令中指定的动作名称
# (.*) 是另一个捕获组，它匹配冒号之后的任意字符，直到字符串结束。这部分用于捕获命令的参数。
action_re = re.compile('^Action: (\w+): (.*)$')

<>:3: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_1292/2417346813.py:3: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')


&emsp;&emsp;测试代码如下：

In [31]:
match = action_re.match("Action: fetch_real_time_info: mass of earth")
if match:
    print(match.group(1))  # 'fetch_real_time_info'
    print(match.group(2))  # 'mass of earth'

fetch_real_time_info
mass of earth


&emsp;&emsp;由此，我们定义了如下的一个 `AgentExecutor`函数。该函数实现一个循环，检测状态并使用正则表达式提取当前停留的状态阶段。不断地迭代，直到没有更多的（或者我们已达到最大迭代次数）调用操作，再返回最终的响应。完整代码如下：

In [32]:
action_re = re.compile('^Action: (\w+): (.*)$')

def AgentExecutor(question, max_turns=5):
    i = 0
    bot = ChatBot(system_prompt)
    # 通过 next_prompt 标识每一个子任务的阶段性输入
    next_prompt = question
    while i < max_turns:
        i += 1
        # 这里调用的就是 ChatBot 类的 __call__ 方法
        result = bot(next_prompt)
        print(f"result:{result}")
        # 在这里通过正则判断是否到了需要调用函数的Action阶段
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # 提取调用的工具名和工具所需的入参
            action, action_input = actions[0].groups()
            if action not in available_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(f"running: {action} {action_input}")
            observation = available_actions[action](action_input)
            print(f"Observation: {observation}")
            next_prompt = "Observation: {}".format(observation)
        else:
            return bot.messages

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_1292/4070349665.py:1: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')


&emsp;&emsp;运行 AI Agent 进行测试：

In [33]:
AgentExecutor("世界上最长的河流是什么？")

result:Thought: 我应该在SerperAPI上查找世界上最长的河流。
Action: fetch_real_time_info: 世界上最长的河流是什么？

running: fetch_real_time_info 世界上最长的河流是什么？
Observation: 1. 尼罗河（Nile）. 6670km ; 2. 亚马逊河（Amazon）. 6400km ; 3. 长江（Chang Jiang）. 6397km ; 4. 密西西比河（Mississippi）. 6020km.
result:Answer: 世界上最长的河流是尼罗河，其长度为6670公里。


[{'role': 'system',
  'content': "You run in a loop of Thought, Action, Observation, Answer.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you.\nObservation will be the result of running those actions.\nAnswer will be the result of analysing the Observation\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nfetch_real_time_info:\ne.g. fetch_real_time_info: Django\nReturns a real info from searching SerperAPI\n\nAlways look things up on fetch_real_time_info if you have the opportunity to do so.\n\nExample session:\n\nQuestion: What is the capital of China?\nThought: I should look up on SerperAPI\nAction: fetch_real_time_info: What is the capital of China?\nPAUSE \n\nYou will be called again with this:\n\nObservation: China is a 

In [34]:
AgentExecutor("20 * 15 等于多少")

result:Thought: I need to calculate the product of 20 and 15.
Action: calculate: 20 * 15
running: calculate 20 * 15
Observation: 300
result:Answer: 20 * 15 等于 300。


[{'role': 'system',
  'content': "You run in a loop of Thought, Action, Observation, Answer.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you.\nObservation will be the result of running those actions.\nAnswer will be the result of analysing the Observation\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nfetch_real_time_info:\ne.g. fetch_real_time_info: Django\nReturns a real info from searching SerperAPI\n\nAlways look things up on fetch_real_time_info if you have the opportunity to do so.\n\nExample session:\n\nQuestion: What is the capital of China?\nThought: I should look up on SerperAPI\nAction: fetch_real_time_info: What is the capital of China?\nPAUSE \n\nYou will be called again with this:\n\nObservation: China is a 

In [35]:
AgentExecutor("世界上最长的河流，与中国最长的河流，它们之间的差值是多少？")

result:Thought: 我需要找到世界上最长的河流和中国最长的河流的长度，并计算它们之间的差值。我应该在SerperAPI上查找这些信息。
Action: fetch_real_time_info: 世界上最长的河流的长度和名称
PAUSE
running: fetch_real_time_info 世界上最长的河流的长度和名称
Observation: 中文名. 世界十大最长河流 ; 外文名. Ten of the world's longest river ; 最长河流. 尼罗河 ; 尼罗河长度. 6670公里.
result:Thought: 我已经找到了世界上最长的河流是尼罗河，长度为6670公里。现在我需要找出中国最长的河流及其长度。
Action: fetch_real_time_info: 中国最长的河流的长度和名称
PAUSE
running: fetch_real_time_info 中国最长的河流的长度和名称
Observation: 天然河流 ; 1, changjiang !长江 · 6300+ ; 2, 黄河, 5464 ; 3, 黑龙江 · 阿穆尔河 · 哈拉穆连 · 3414 !3414 （4416） ; 4, 塔里木河, 2572 ...
result:Thought: 我获取到了中国最长的河流是长江，长度为6300公里。现在我可以计算尼罗河和长江之间的长度差。
Action: calculate: 6670 - 6300
PAUSE
running: calculate 6670 - 6300
Observation: 370
result:Answer: 世界上最长的河流尼罗河与中国最长的河流长江之间的长度差为370公里。


[{'role': 'system',
  'content': "You run in a loop of Thought, Action, Observation, Answer.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you.\nObservation will be the result of running those actions.\nAnswer will be the result of analysing the Observation\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nfetch_real_time_info:\ne.g. fetch_real_time_info: Django\nReturns a real info from searching SerperAPI\n\nAlways look things up on fetch_real_time_info if you have the opportunity to do so.\n\nExample session:\n\nQuestion: What is the capital of China?\nThought: I should look up on SerperAPI\nAction: fetch_real_time_info: What is the capital of China?\nPAUSE \n\nYou will be called again with this:\n\nObservation: China is a 

&emsp;&emsp;从上面我们实现的案例中，非常明显的发现，ReAct（推理和行动）框架通过将推理和行动整合到一个有凝聚力的操作范式中，能够实现动态和自适应问题解决，从而允许与用户和外部工具进行更复杂的交互。这种方法不仅增强了大模型处理复杂查询的能力，还提高了其在多步骤任务中的性能，使其适用于从自动化客户服务到复杂决策系统的广泛应用。

&emsp;&emsp;就目前的AI Agent 现状而言，流行的代理框架都有内置的 ReAct 代理，比如`Langchain`、`LlamaIndex`中的代理，或者 `CrewAI`这种新兴起的AI Agent开发框架，都是基于ReAct理念的一种变种。LangChain 的 ReAct 代理工程描述 👇

```json
Answer the following questions as best you can. You have access 
to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

There are three placeholders {tool}, {input}, and {agent_scratchpad} in this prompt. These will be replaced with the appropriate text before sending it to LLM.
```

&emsp;&emsp;这个提示中有三个占位符 {tool}、{input} 和 {agent_scratchpad}。在发送给LLM之前，这些内容将被替换为适当的文本。
- tools - 工具的描述
- tool_names - 工具的名称
- input - 大模型接收的原始问题（通常是来自用户的问题）
- agent_scratchpad - 保存以前的想法/行动/行动输入/观察的历史记录

&emsp;&emsp;因此，基于 `ReAct` 的代理工程并非一成不变，其所调用的工具也不局限于单一类型。这种灵活性实际上为 `AI Agent` 执行人工智能代理任务开启了无限的可能性。